## Single label

In [89]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [91]:
x = np.genfromtxt ('C:/Users/z5023853/Google Drive/Separated dataset_ML/a1_TCtrain.csv', 
                    delimiter = ',', usecols = (4,5,6), skip_header=1)
y = np.genfromtxt ('C:/Users/z5023853/Google Drive/Separated dataset_ML/a1_TCtrain.csv', 
                   delimiter =',', usecols = (7),skip_header=1)
x_cv = np.genfromtxt ('C:/Users/z5023853/Google Drive/Separated dataset_ML/a1_TCcv.csv', 
                    delimiter = ',', usecols = (4,5,6), skip_header=1)
y_cv = np.genfromtxt ('C:/Users/z5023853/Google Drive/Separated dataset_ML/a1_TCcv.csv', 
                   delimiter =',', usecols = (7),skip_header=1)
test_dataset = np.genfromtxt ('C:/Users/z5023853/Google Drive/Separated dataset_ML/a1_TCcv.csv', 
                              delimiter = ',', usecols = (4,5,6,7), skip_header=1)
train_dataset = np.genfromtxt ('C:/Users/z5023853/Google Drive/Separated dataset_ML/a1_TCtrain.csv', 
                              delimiter = ',', usecols = (4,5,6,7), skip_header=1)

In [92]:
y_train = np.reshape(y, (54,1)).astype(np.float32)
y_test = np.reshape (y_cv, (14,1)).astype(np.float32)

In [94]:
print (x.shape)
print (x_cv.shape)
print (x_whole.shape)
print (y.shape)
print (y_cv.shape)
x_whole = np.vstack ((x, x_cv)).astype(np.float32)

(54, 3)
(14, 3)
(68, 3)
(54,)
(14,)


In [95]:
enc = OneHotEncoder ()
x_enc = enc.fit_transform (x_whole).toarray().astype(np.float32)
x_train = x_enc [:54, :]
x_test = x_enc [54:, :]
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)
#print (x_train)
#print (x_test)
#train_dataset = np.hstack ((x_train, y)).astype(np.float32)
#test_dataset = np.hstack ((x_test, y_cv)).astype(np.float32)
print (train_dataset.shape)
print (test_dataset.shape)

(54, 20)
(14, 20)
(54, 1)
(14, 1)
(54, 4)
(14, 4)


### Building models

In [96]:
hidden_nodes = 10
num_labels = y_train.shape[1]
batch_size = 100
num_features = x_train.shape[1]
learning_rate = .01

In [97]:
graph = tf.Graph()
with graph.as_default():
    
    # Data
    tf_train_dataset = tf.placeholder(tf.float32, shape = [None, num_features])
    tf_train_labels = tf.placeholder(tf.float32, shape = [None, num_labels])
    tf_test_dataset = tf.constant(x_test)
  
    # Weights and Biases
    layer1_weights = tf.Variable(tf.truncated_normal([num_features, hidden_nodes]))
    layer1_biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    layer2_biases = tf.Variable(tf.zeros([num_labels]))
    
    # Three-Layer Network
    def three_layer_network (data):
        input_layer = tf.matmul(data, layer1_weights)
        hidden = tf.nn.relu(input_layer + layer1_biases)
        output_layer = tf.matmul(hidden, layer2_weights) + layer2_biases
        return output_layer
    
    # Model Scores
    model_scores = three_layer_network(tf_train_dataset)
    
    # Loss
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = model_scores, labels = tf_train_labels))
  
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(model_scores)
    test_prediction = tf.nn.softmax(three_layer_network(tf_test_dataset))

In [98]:
def accuracy(predictions, labels):
    preds_correct_boolean =  np.argmax(predictions, 1) == np.argmax(labels, 1)
    correct_predictions = np.sum(preds_correct_boolean)
    accuracy = 100.0 * correct_predictions / predictions.shape[0]
    return accuracy

In [99]:
print (model_scores)

Tensor("add_1:0", shape=(?, 1), dtype=float32)


In [70]:
#this is a mini batch gradient descent
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (y_train.shape[0] - batch_size)
        minibatch_data = x_train[offset:(offset + batch_size), :]
        minibatch_labels = y_train[offset:(offset + batch_size)]
        
        feed_dict = {tf_train_dataset : minibatch_data, tf_train_labels : minibatch_labels}
        
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict = feed_dict
            )
        
        if step % 1000 == 0:
            print ("Minibatch loss at step: {0} {1}".format (step, 1))
    print ('Test accuracy: {0}%'.format(accuracy(test_prediction.eval(), y_test)))

Minibatch loss at step: 0 1
Minibatch loss at step: 1000 1
Minibatch loss at step: 2000 1
Minibatch loss at step: 3000 1
Minibatch loss at step: 4000 1
Minibatch loss at step: 5000 1
Minibatch loss at step: 6000 1
Minibatch loss at step: 7000 1
Minibatch loss at step: 8000 1
Minibatch loss at step: 9000 1
Minibatch loss at step: 10000 1
Test accuracy: 100.0%


In [71]:
layer1_weights
#print (layer2_weights)

<tf.Variable 'Variable:0' shape=(20, 10) dtype=float32_ref>

In [72]:
print (first_layer_neurons)

Tensor("xw_plus_b:0", shape=(54, 10), dtype=float32)


### Gradient descent

In [73]:
training_data = x_train
training_labels = y_train

In [77]:
hidden_nodes = 5
num_labels = training_labels.shape[1]
num_features = training_data.shape[1]
learning_rate = .01
reg_lambda = .01

In [75]:
# Weights and Bias Arrays, just like in Tensorflow
layer1_weights_array = np.random.normal(0, 1, [num_features, hidden_nodes]) 
layer2_weights_array = np.random.normal(0, 1, [hidden_nodes, num_labels]) 
layer1_biases_array = np.zeros((1, hidden_nodes))
layer2_biases_array = np.zeros((1, num_labels))

In [82]:
#Relu activation
def relu_activation(data_array):
    return np.maximum(data_array, 0)

#Softmax activation
def softmax(output_array):
    logits_exp = np.exp(output_array)
    return logits_exp / np.sum(logits_exp, axis = 1, keepdims = True)

#softmax cross entropy
def cross_entropy_softmax_loss_array(softmax_probs_array, y_onehot):
    indices = np.argmax(y_onehot, axis = 1).astype(int)
    predicted_probability = softmax_probs_array[np.arange(len(softmax_probs_array)), indices]
    log_preds = np.log(predicted_probability)
    loss = -1.0 * np.sum(log_preds) / len(log_preds)
    return loss

#Regularisation of softmax
def regularization_L2_softmax_loss(reg_lambda, weight1, weight2):
    weight1_loss = 0.5 * reg_lambda * np.sum(weight1 * weight1)
    weight2_loss = 0.5 * reg_lambda * np.sum(weight2 * weight2)
    return weight1_loss + weight2_loss

In [86]:
for step in range(5001):

    input_layer = np.dot(training_data, layer1_weights_array)
    hidden_layer = relu_activation(input_layer + layer1_biases_array)
    output_layer = np.dot(hidden_layer, layer2_weights_array) + layer2_biases_array
    output_probs = softmax(output_layer)
    
    loss = cross_entropy_softmax_loss_array(output_probs, training_labels)
    loss += regularization_L2_softmax_loss(reg_lambda, layer1_weights_array, layer2_weights_array)

    output_error_signal = (output_probs - training_labels) / output_probs.shape[0]
    
    error_signal_hidden = np.dot(output_error_signal, layer2_weights_array.T) 
    error_signal_hidden[hidden_layer <= 0] = 0
    
    gradient_layer2_weights = np.dot(hidden_layer.T, output_error_signal)
    gradient_layer2_bias = np.sum(output_error_signal, axis = 0, keepdims = True)
    
    gradient_layer1_weights = np.dot(training_data.T, error_signal_hidden)
    gradient_layer1_bias = np.sum(error_signal_hidden, axis = 0, keepdims = True)

    gradient_layer2_weights += reg_lambda * layer2_weights_array
    gradient_layer1_weights += reg_lambda * layer1_weights_array

    layer1_weights_array -= learning_rate * gradient_layer1_weights
    layer1_biases_array -= learning_rate * gradient_layer1_bias
    layer2_weights_array -= learning_rate * gradient_layer2_weights
    layer2_biases_array -= learning_rate * gradient_layer2_bias
    
    if step % 500 == 0:
            print ('Loss at step {0}: {1}'.format(step, loss))

c:\users\z5023853\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in maximum
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\z5023853\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in less_equal
  


Loss at step 0: nan
Loss at step 500: nan
Loss at step 1000: nan
Loss at step 1500: nan
Loss at step 2000: nan
Loss at step 2500: nan
Loss at step 3000: nan
Loss at step 3500: nan
Loss at step 4000: nan
Loss at step 4500: nan
Loss at step 5000: nan


In [88]:
input_layer = np.dot(test_dataset, layer1_weights_array)
hidden_layer = relu_activation(input_layer + layer1_biases_array)
scores = np.dot(hidden_layer, layer2_weights_array) + layer2_biases_array
probs = softmax(scores)
print ('Test accuracy: {0}%'.format(accuracy(probs, test_labels)))

ValueError: shapes (14,21) and (20,5) not aligned: 21 (dim 1) != 20 (dim 0)